In [ ]:
import os
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from skimage import io, color, transform, feature
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Path to the folders containing real and fake images
original_folder_path = "/content/gdrive/MyDrive/celebdf/original-cropped-images"
synthetic_folder_path = "/content/gdrive/MyDrive/celebdf/synthetic-cropped-images"

In [ ]:
# Load and preprocess images
def load_and_preprocess_images(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            img = io.imread(img_path)
            img_gray = color.rgb2gray(img)
            img_resized = transform.resize(img_gray, (64, 64))  # Resize to a consistent size
            images.append(img_resized)
    return images

In [ ]:
# Load and preprocess real and fake images
fake_images = load_and_preprocess_images(synthetic_folder_path)
real_images = load_and_preprocess_images(original_folder_path)

# Create labels (0 for real, 1 for fake)
real_labels = np.zeros(len(real_images))
fake_labels = np.ones(len(fake_images))

In [ ]:
# Combine real and fake data
all_images = np.concatenate((real_images, fake_images), axis=0)
all_labels = np.concatenate((real_labels, fake_labels), axis=0)


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

# Parameters for LBP
radius = 1
n_points = 8 * radius

# Extract LBP features
def extract_lbp_features(images):
    features = []
    for img in images:
        lbp_img = local_binary_pattern(img, n_points, radius, method='uniform')
        hist, _ = np.histogram(lbp_img.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-8)
        features.append(hist)
    return features

X_train_features = np.array(extract_lbp_features(X_train))
X_test_features = np.array(extract_lbp_features(X_test))


In [ ]:
# Define your neural network architecture
def build_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = Dense(512, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
feature_dim = X_train_features.shape[1]
model = build_model((feature_dim,))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
batch_size = 32
epochs = 10

model.fit(X_train_features, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_features, y_test))

Epoch 1/10
750/750 [==============================] - 15s 8ms/step - loss: 0.5387 - accuracy: 0.7844 - val_loss: 0.3599 - val_accuracy: 0.8827
Epoch 2/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3991 - accuracy: 0.8717 - val_loss: 0.3554 - val_accuracy: 0.8827
Epoch 3/10
750/750 [==============================] - 6s 8ms/step - loss: 0.3841 - accuracy: 0.8752 - val_loss: 0.3537 - val_accuracy: 0.8827
Epoch 4/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3792 - accuracy: 0.8764 - val_loss: 0.3518 - val_accuracy: 0.8827
Epoch 5/10
750/750 [==============================] - 5s 7ms/step - loss: 0.3733 - accuracy: 0.8769 - val_loss: 0.3519 - val_accuracy: 0.8827
Epoch 6/10
750/750 [==============================] - 6s 8ms/step - loss: 0.3711 - accuracy: 0.8769 - val_loss: 0.3513 - val_accuracy: 0.8827
Epoch 7/10
750/750 [==============================] - 6s 8ms/step - loss: 0.3700 - accuracy: 0.8769 - val_loss: 0.3507 - val_accuracy: 0.8827
Epoch

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_features, y_test)
print("Test accuracy:", test_accuracy)

188/188 [==============================] - 1s 4ms/step - loss: 0.3479 - accuracy: 0.8829
Test accuracy: 0.8828828930854797
